In [1]:
import evebox as box
import pandas as pd
import tensorflow as tf
import cProfile
from tqdm.notebook import tqdm, trange

In [2]:
uni = box.Universe.from_esi(cache = 'universe.txt', tqdm = tqdm).range('Jita', min_sec = 0.9).with_market_types()

In [3]:
#orders = box.load_orders(uni, tqdm = tqdm)
orders = pd.read_csv('market/current.csv')

In [4]:
#orders.to_csv('market/current.csv')

In [5]:
orders

,order_id,duration,is_buy_order,issued,location_id,min_volume,price,range,system_id,type_id,volume_remain,volume_total
0,911203062,365,False,2020-01-02T11:04:27+00:00,60000298,1,50000.00,region,30000129,967,1,1
1,911203074,365,False,2020-03-08T14:36:26+00:00,60000298,1,109.02,region,30000129,3673,94161,94161
2,911203098,365,True,2020-03-08T14:25:05+00:00,60000298,1,208.99,station,30000129,3717,13606,13606
3,911203190,365,False,2020-01-23T11:09:24+00:00,60000358,1,50000.00,region,30000144,967,1,1
4,911203191,365,False,2020-02-03T11:06:18+00:00,60000361,1,50000.00,region,30000142,967,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
258434,5628621294,14,False,2020-03-08T15:09:36+00:00,60015027,1,9.00,region,30030141,34,7250,7250
258435,5628621295,90,True,2020-03-08T15:09:37+00:00,1028858195912,1,1830000.09,1,30000144,2195,20,20
258436,5628621307,30,False,2020-03-08T15:09:38+00:00,60003760,1,99888.98,region,30000142,20420,2000,2000
258437,5628621309,90,False,2020-03-08T15:09:38+00:00,60003760,1,1399.79,region,30000142,27447,100000,100000


In [6]:
gym = box.tf.TradingGym(uni, orders)

In [7]:
jita = [s["system_id"] for s in uni.systems.values() if s["name"] == "Jita"][0]

state = box.MutableState()
state.time_left = 100.0
state.system    = jita

state = box.State(state)

In [8]:
model = box.tf.ReferenceModel(uni, orders, state_shape = [64])

unroller = gym.unroll_model(model, tqdm)

In [9]:
with tf.GradientTape() as tape:
    #result = unroller(
    #    state, 150
    #)
    cProfile.run('unroller(state, 100)', 'profile_stats')

Reference model called
({'state': <tf.Tensor 'args_0_4:0' shape=(1,) dtype=float32>, 'systems': <tf.Tensor 'args_0_5:0' shape=(16, 2) dtype=float32>, 'cargo': <tf.Tensor 'args_0:0' shape=(14472, 2) dtype=float32>, 'orders': (<tf.Tensor 'args_0_1:0' shape=(258439,) dtype=int32>, <tf.Tensor 'args_0_2:0' shape=(258439,) dtype=int32>, <tf.Tensor 'args_0_3:0' shape=(258439, 3) dtype=float32>)}, <tf.Tensor 'self:0' shape=(64,) dtype=float32>)
Reference model called
({'state': <tf.Tensor 'args_0_4:0' shape=(1,) dtype=float32>, 'systems': <tf.Tensor 'args_0_5:0' shape=(16, 2) dtype=float32>, 'cargo': <tf.Tensor 'args_0:0' shape=(14472, 2) dtype=float32>, 'orders': (<tf.Tensor 'args_0_1:0' shape=(258439,) dtype=int32>, <tf.Tensor 'args_0_2:0' shape=(258439,) dtype=int32>, <tf.Tensor 'args_0_3:0' shape=(258439, 3) dtype=float32>)}, <tf.Tensor 'self:0' shape=(64,) dtype=float32>)
Reference model called
({'state': <tf.Tensor 'args_0_4:0' shape=(1,) dtype=float32>, 'systems': <tf.Tensor 'args_0_5:0

In [10]:
import pstats
p = pstats.Stats('profile_stats')
p.sort_stats('cumtime')
p.print_stats()

Mon Mar  9 09:43:34 2020    profile_stats

         7276502 function calls (6762313 primitive calls) in 11.454 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    121/1    0.033    0.000   11.462   11.462 {built-in method builtins.exec}
        1    0.005    0.005   11.461   11.461 D:\Daten\Repos\tf_playground\eveplan\evebox\tf\trading_gym.py:221(run)
      101    0.007    0.000    7.090    0.070 D:\Daten\Repos\tf_playground\eveplan\evebox\tf\trading_gym.py:129(encode_state)
      933    0.003    0.000    6.191    0.007 d:\programme\python 3.6.6\lib\site-packages\pandas\core\indexing.py:1753(__getitem__)
      101    0.003    0.000    6.091    0.060 d:\programme\python 3.6.6\lib\site-packages\pandas\core\indexing.py:1901(_getitem_axis)
      101    0.001    0.000    6.084    0.060 d:\programme\python 3.6.6\lib\site-packages\pandas\core\indexing.py:1556(_getitem_iterable)
      202    0.003    0.000    6.079    0.030 d:\pr

In [11]:
result[0:1]

NameError: name 'result' is not defined

In [ ]:
tape.gradient(result[0][2], sources=model.trainable_variables)

In [ ]:
model.trainable_variables